In [1]:
using LinearAlgebra, Distributions, Statistics, Plots, KernelFunctions, Random, Interpolations, ForwardDiff, Gridap, NPZ, ProgressMeter
import Gridap: ∇

In [2]:
using IJulia

# Elliptic ODE with parametrised diffusion coefficient:

$\newcommand{R}{\mathbb{R}}$

\begin{align}
-\frac{\mathrm{d}}{\mathrm{d}x}\left(\kappa(x;p)\frac{\mathrm{d}}{\mathrm{d}x} u(x;p)\right) &= f(x), \quad x\in\Omega:=[0,1] \\
u(x) &= 0, \quad x\in\partial\Omega
\end{align}

where: $\kappa:\R\times\R^{d}\rightarrow\R$ is defined by:

$$
\log\kappa(x;p) := \sum_{k=1}^{d} p_{k} \lambda_{k} \phi_{k}(x)
$$

where the the pairs $\{\lambda_{k},\phi_{k}(x)\}$ are the eigenpairs of the correlation operator associated with the Matern-1/2 covariance function:

$$
c(x,y) = \exp\left(-\beta^{-1}|x-y|\right)
$$

where we truncate the KL-expansion at $d=100$ terms so our parameter vector $p\in\R^{100}$.

Note here the $\{p_{k}\}_{k=1}^{d}$ are i.i.d. $\mathcal{N}(0,1)$ random variables

In [3]:
function eigenpairs(β,d)
    xs = range(0, stop=1, length=d)
    
    # form kernel and kernel mat
    c = with_lengthscale(Matern12Kernel(), β)
    K = kernelmatrix(c, xs)

    # get spectrum
    spectrum = eigen(K, sortby = (x -> -real(x)))
    
    # form eigenpairs
    Φ = √(d) * spectrum.vectors
    ϕ(x::Float64,i::Int) = linear_interpolation(xs, Φ[:,i])(x)
    λ = spectrum.values / d
    return ϕ, λ
end

eigenpairs (generic function with 1 method)

In [4]:
function κ(x, p, ϕ, λ)
    d = length(p)
    # compute log_κ and κ
    log_κ = sum(p .* λ .* ϕ.(x, 1:d))
    return exp(log_κ)
end

κ (generic function with 1 method)

In [22]:
Random.seed!(42)
d = 100
p = rand(Normal(), d)
β = 0.5
ϕ, λ_KL = eigenpairs(β, d)
κ(0.5, p, ϕ, λ_KL)

1.287367884787411

In [7]:
function dκdp(x,p,ϕ,λ,j)
    d = length(p)
    log_κ = sum(p .* λ .* ϕ.(x, 1:d))
    return λ[j] * ϕ(x,j) * exp(log_κ)
end

dκdp (generic function with 1 method)

In [20]:
dκdp(0.5,p,ϕ,λ,2)

2.1074261117499784e-16

In [23]:
function dudp(p, ϕ, λ_KL, κ, dκdp, x0, f, n=50, order=1)
    # set up mesh and spaces
    domain = (0,1)
    partition = (n)
    model = CartesianDiscreteModel(domain, partition)

    reffe = ReferenceFE(lagrangian,Float64,order)
    V0 = TestFESpace(model,reffe,conformity=:H1,dirichlet_tags="boundary")
    U = TrialFESpace(V0,x->0)

    degree = order + 1

    # solve for FEM solution uh of original problem (forward pass)
    Ω = Triangulation(model)
    dΩ = Measure(Ω,degree)

    kappa(x::VectorValue) = κ(x[1],p, ϕ, λ_KL)
    a(u,v) = ∫( kappa * ∇(v)⊙∇(u) )*dΩ
    b(v) = ∫( v*f )*dΩ

    op = AffineFEOperator(a,b,U,V0)
    uh = solve(op)

    # solve for adjoint variable λ (back pass)
    δ = DiracDelta(model, Point(x0))
    b_λ(v) = -1*δ(v)
    
    op_λ = AffineFEOperator(a,b_λ,U,V0)
    λ = solve(op_λ)

    # compute derivative for each parameter and return and return
    grads = []
    d = length(p)
    for j ∈ 1:d
        dkappa(x::VectorValue) = dκdp(x[1],p,ϕ,λ_KL,j)
        sens = sum( ∫( dkappa * ∇(λ)⋅∇(uh) )*dΩ )
        push!(grads, sens)
    end
    
    return grads
end

dudp (generic function with 3 methods)

In [25]:
f(x) = 1.0
x0 = 0.5
Random.seed!(42)
d = 100
p = rand(Normal(), d)
β = 0.5
ϕ, λ_KL = eigenpairs(β, d)

(ϕ, [0.5720412361334664, 0.19618886458384138, 0.07912283116165131, 0.040154110559142364, 0.023814405436660913, 0.01564704630248294, 0.011031950775377835, 0.008184913417153977, 0.006310315651440851, 0.005012664929596786  …  0.00010353969216468372, 0.00010305195042530761, 0.00010261814462574573, 0.00010223739531059515, 0.00010190893513120877, 0.00010163210553466771, 0.00010140635394149013, 0.00010123123139176252, 0.00010110639064216426, 0.00010103158469992617])

In [26]:
# dudp(p, ϕ, λ_KL, κ, dκdp, x0, f)

100-element Vector{Any}:
  0.05542587400569864
 -0.0009683825648324982
 -0.005834004498755138
  3.819247779626988e-5
  0.00020629754143597324
 -2.4691387334250636e-5
  7.126858324129904e-5
 -1.2867695581950692e-6
 -1.3857352213645639e-5
 -5.790744057988898e-6
 -1.3599150479765641e-6
 -7.0269001032017e-7
  5.751980901336189e-6
  ⋮
  2.1909916562519897e-7
  2.2096312865053965e-8
  2.2392988877670717e-7
  1.9752239323708825e-9
  3.905336592951781e-7
  1.993041481386328e-8
  4.9357412090441e-7
 -3.6958895798620365e-10
  8.082027182219375e-7
  3.4727966277200546e-8
  1.347013334767501e-6
  5.879710562071544e-8

In [27]:
@time begin
    grads = dudp(p, ϕ, λ, κ, dκdp, x0, f);
end

  0.580155 seconds (3.70 M allocations: 2.865 GiB, 27.37% gc time)


100-element Vector{Any}:
  0.05542587400569864
 -0.0009683825648324982
 -0.005834004498755138
  3.819247779626988e-5
  0.00020629754143597324
 -2.4691387334250636e-5
  7.126858324129904e-5
 -1.2867695581950692e-6
 -1.3857352213645639e-5
 -5.790744057988898e-6
 -1.3599150479765641e-6
 -7.0269001032017e-7
  5.751980901336189e-6
  ⋮
  2.1909916562519897e-7
  2.2096312865053965e-8
  2.2392988877670717e-7
  1.9752239323708825e-9
  3.905336592951781e-7
  1.993041481386328e-8
  4.9357412090441e-7
 -3.6958895798620365e-10
  8.082027182219375e-7
  3.4727966277200546e-8
  1.347013334767501e-6
  5.879710562071544e-8

In [21]:
function getAS(x0, ps, ϕ, λ_KL, κ, dκdp, f)
    # compute gradients at each p in ps
    Grads = map(p -> dudp(p, ϕ, λ_KL, κ, dκdp, x0, f), eachcol(ps))

    # take outer product of each grad with itself
    C_x = map(grad -> grad * grad', Grads)

    # compute mean of outer products
    C_hat = Statistics.mean(C_x)

    # eigen decomposition of C_hat
    spectrum = eigen(C_hat)
    λ = spectrum.values

    # compute proportion of spectrum explained by max eigenvector
    λ_max = max(λ...)
    prop = λ_max / sum(abs,λ)

    # get AS as leading eigenvector
    W = spectrum.vectors[:, argmax(λ)]
    @assert W' * W ≈ 1.0

    return W, λ, prop
end

getAS (generic function with 1 method)

In [22]:
Random.seed!(42)
d = 50
M = 100
ps = rand(Normal(), (d,M));

In [23]:
@time begin
    getAS(0.5,ps,β,κ,dκdp,f)
end

320.544404 seconds (148.93 M allocations: 156.736 GiB, 5.48% gc time, 1.17% compilation time)


([-0.9993715507868105, -7.209616069710391e-5, -0.03542661971111379, 0.0003883350419153888, 0.0010960590364227014, -4.423210893091799e-6, 0.00012450450582100035, -9.705869889396412e-6, 0.00023854459809929094, 7.66247121627373e-6  …  5.120062711017431e-6, 1.0784746229077707e-7, 3.939566959042385e-6, 7.17366796077978e-9, -2.934851769385168e-6, -7.752174241624597e-8, -1.8125025771997835e-6, 2.5595469632345005e-8, -8.817735712707275e-7, -4.935447499440276e-8], [2.918474682749087e-20, 4.0312776458955875e-20, 8.710966584861892e-20, 1.5184287905589437e-19, 2.053651216177132e-19, 2.3978093412191173e-19, 5.54741102481446e-19, 6.512186035900109e-19, 7.203725212376894e-19, 8.011764666917357e-19  …  1.5460925990246412e-12, 6.7988799469124835e-12, 1.970018240800247e-11, 5.231795276075645e-11, 2.7701692904475334e-10, 1.624314700556985e-9, 3.012249782429064e-8, 1.7047012084003114e-7, 5.206748546997236e-6, 0.02015457194358601], 0.9997316801110835)

In [27]:
((321 * 3 * 40) / 60) / 60

10.7

In [10]:
for (i,x) in enumerate(xs)
    IJulia.clear_output(true)
    @show i
end

LoadError: UndefVarError: `xs` not defined

In [11]:
β = 1.0

1.0

In [34]:
(((6*100)/60)*5)

50.0

In [38]:
xs = range(0.01,stop=0.99,length=5)

0.01:0.245:0.99

In [ ]:
AS_results = []
for (i,x) ∈ enumerate(xs)
    IJulia.clear_output(true)
    @show i
    result = getAS(x, ps, β, κ, dκdp, f)
    push!(AS_results, result)
end

i = 3


In [40]:
# AS_results = @showprogress map(x -> getAS(x, ps, β, κ, dκdp, f), xs);

In [29]:
AS_results = [(1,2,4),(3,4,-1.0)]

2-element Vector{Tuple{Int64, Int64, Real}}:
 (1, 2, 4)
 (3, 4, -1.0)

In [30]:
using JLD

In [31]:
save("test.jld", "AS_results", AS_results)

In [32]:
res = load("test.jld")

Dict{String, Any} with 1 entry:
  "AS_results" => Tuple{Int64, Int64, Real}[(1, 2, 4), (3, 4, -1.0)]

In [34]:
res["AS_results"]

2-element Vector{Tuple{Int64, Int64, Real}}:
 (1, 2, 4)
 (3, 4, -1.0)